In [ ]:
import numpy as np
import keras
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
import time, os, fnmatch, shutil

In [ ]:
cmmd_dir = "/media/craig/Larry/python/"

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        preprocessing_function=tf.keras.applications.vgg16.preprocess_input)
test_datagen = ImageDataGenerator(rescale=1./255,
                                 preprocessing_function=tf.keras.applications.vgg16.preprocess_input)
train_generator = train_datagen.flow_from_directory(
        cmmd_dir+"subset_two/PNG/TRAIN/",
        color_mode='rgb',
        target_size=(224, 224),
        batch_size=32,
        shuffle=True,
        class_mode='binary',
        seed=11)
validation_generator = test_datagen.flow_from_directory(
        cmmd_dir+"subset_two/PNG/VAL/",
        color_mode='rgb',
        target_size=(224, 224),
        batch_size=32,
        shuffle=True,
        class_mode='binary',
        seed=11)

# VGG16 Transfer Learning

In [ ]:
vgg16_base = tf.keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

In [ ]:
model = tf.keras.models.Sequential()
model.add(vgg16_base)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.layers[0].trainable = False

In [ ]:
import time, os, fnmatch, shutil
root_logdir = os.path.join(os.curdir, "my_logs")

def get_run_logdir():
    #TensorBaoard run TITLE:
    run_id = ("VGG16_Transfer_Learning")
    return os.path.join(root_logdir,run_id)

run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

callbacks = [
    keras.callbacks.ModelCheckpoint("/media/craig/Henry/ML_models/vgg16/save_at_{epoch}.h5"),
    keras.callbacks.TensorBoard(run_logdir)
]

In [ ]:
history = model.fit(
    train_generator,
    epochs=500,
    validation_data=validation_generator,
    callbacks=callbacks
)

In [ ]:
model.save('/media/craig/Henry/ML_models/vgg16/final_model_epoch_500.h5')

In [ ]:
loss_hist = history.history['val_loss']

lowest_val_loss = np.min(loss_hist)
best_epochs = np.argmin(loss_hist) + 1
print("The lowest validation loss is: "+str(lowest_val_loss)+"\nAt epoch: "+str(best_epochs))

bin_acc_hist = history.history['val_accuracy']

lowest_val_loss = np.max(bin_acc_hist)
best_epochs = np.argmax(bin_acc_hist) + 1
print("The highest validation acc is: "+str(lowest_val_loss)+"\nAt epoch: "+str(best_epochs))